# NNIA Assignment 9

**DEADLINE: 18. 01. 2023 08:00 CET**
Submission more than 10 minutes past the deadline will **not** be graded!

- Name & ID 1 (Teams username e.g. s8xxxxx):
- Name & ID 2 (Teams username e.g. s8xxxxx):
- Name & ID 3 (Teams username e.g. s8xxxxx):
- Hours of work per person:

# Submission Instructions

**IMPORTANT** Please make sure you read the following instructions carefully. If you are unclear about any part of the assignment, ask questions **before** the assignment deadline. All course-related questions can be addressed on the course **[Piazza Platform](https://piazza.com/class/kvc3vzhsvh55rt)**.

* Assignments are to be submitted in a **team of 2 or 3**.
* Please include your **names**, **ID's**, **Teams usernames**, and **approximate total time spent per person** at the beginning of the Notebook in the space provided
* Make sure you appropriately comment your code wherever required.
* Your final submission should contain this completed Jupyter Notebook, including the bonus question (if you attempt it), and any necessary Python files.
* Do **not** submit any **data or cache files** (e.g. `__pycache__`, the dataset PyTorch downloads, etc.). 
* Upload the **zipped** folder (*.zip* is the only accepted extension) in **Teams**.
* Only **one member** of the group should make the submisssion.
* **Important** please name the submitted zip folder as: `Name1_id1_Name2_id2.zip`. The Jupyter Notebook should also be named: `Name1_id1_Name2_id2.ipynb`. This is **very important** for our internal organization epeatedly students fail to do this.

## 1 Theory Review (3 pts)



Review [chapter 9](https://www.deeplearningbook.org/contents/convnets.html) of the Deep Learning book and the lecture slides and answer the following questions: 
1. In your own words, how are the concepts of *sparse interactions*, *parameter sharing*, and *equivariant representations* applied in convolutional neural networks? *(1 pt)*
1. What kinds of (zero-)padding schemes are there (as decribed in the book)? When should one scheme be chosen over another? (1 pt)
1. How do CNNs handle inputs of varrying sizes? (0.5 pts)
1. What are learned invariances and what effect do they have on the number of kernel parameters? (0.5 pts)

## <font color="red">To Do</font>


### 1

## 2 Output of a convolutional layer (3 pts)
The following [blog post](https://www.baeldung.com/cs/convolutional-layer-size) may be helpful for this exercise.  
Compute the output of a convonlutional layer given the kernel:  
\begin{array}{|l|l|}
\hline
1 & 0 & 1 \\ \hline
0 & 1 & 0 \\ \hline
1 & 0 & 1 \\ \hline
\end{array}  
and the following input RGB image with three values in each cell - one for each channel:  
  
\begin{array}{|c|c|c|c|}
\hline
1, 2, 3 & 1, 1, 0 & 2, 0, 1 & 1, 1, 0 \\ \hline
0, 0, 0 & 1,3,1 & 0, 2, 1 & -1, -2, 0 \\ \hline
-1, 0, 0 & 1, 1, 2 & 0, 0, 0 & 1, 1, 0 \\ \hline
1, 1, 1 & 1, 0, 2 & 1, 3, -3 & 0, 0, 0 \\ \hline
\end{array}  
Use stride = 1 and padding = 0 and state the output dimensionality. What would the output dimensionality be if we used padding = 1?  

## <font color="red">To Do</font>


### 2

## 3 Implementation (4 pts)

### Prelude -- Using a validation dataset

In this task we also officially introduce the use of a validation set in a homework assignment.

A validation set allows you to due some further fine-tuning of your hyperparameters on a set of data that was not used for training. This has two purposes: 1) testing the generalization capabilities of your model and 2) verify that your model can handle unseen data as this may come in formats that you were not specifically expecting.

Take a look at these articles to gain more insight into why we use validation sets in the world of Deep Learning:

* [Why use both validation set and test set?](https://datascience.stackexchange.com/a/18346)
* [Why Do We Need a Validation Set in Addition to Training and Test Sets?](https://towardsdatascience.com/why-do-we-need-a-validation-set-in-addition-to-training-and-test-sets-5cf4a65550e0)

### Implementation task
In this exercise, we will continue to work with [the PyTorch Datasets Class](https://pytorch.org/vision/stable/datasets.html) to obtain
[the CIFAR100 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html). Instead of the simple neural network from the previous assignment, we are going to implement a convolution neural network (CNN) model to classify the images in this dataset into their proper classes.

Your CNN model will have the following architecture:


* It will have five convolution blocks. 
* Each block consists of the *convolution*, *max pooling* and *ReLU* operation in that order. 
* We will use $3\times3$ kernels in all convolutional layers. Set the padding and stride of the convolutional layers so that they **maintain** the spatial dimensions. 
* Max pooling operations are done with $2\times2$ kernels, with a stride of 2, thereby **halving** the spatial resolution each time. 
* Finally, five stacking these five blocks leads to a $512\times1\times1$ feature map. 
* Classification is achieved using a fully connected layer. 

Implement the class *ConvNet* to define the model described. The ConvNet model takes $32\times32$ color images as inputs and has 5 hidden layers with 128, 512, 512, 512, 512 filters, and produces a 100-class classification. We will train the convolutional neural network on the CIFAR-100 dataset. Feel free to incorporate drop-put, batch normalization, and early stopping if desired. Evaluate your trained model on the test set and report your findings.

For loss, you can use cross entropy loss and for optimization, you can use the Adam optimizer with the learning rate of `2e-3` and weight decay of $0.001$. 
       
**Note**: To speed up trainining on the entire dataset, you may want an access to a GPU (CPU runtime > 10 hrs vs < 5 mins GPU). We recommend you make use of [Google Colab](https://colab.research.google.com/?utm_source=scs-index).  
If you are having trouble loading the dataset [this post](https://stackoverflow.com/questions/71263622/sslcertverificationerror-when-downloading-pytorch-datasets-via-torchvision) on stackoverflow may be helpful.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Import some libraries
import torch
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from pathlib import Path

In [ ]:
# We recommend using a GPU for this task, if not available a CPU will be used
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Feel free to modify these variables how you deem fit,
# as long as you are still following the above instructions
# (e.g. you respect the proposed network architecture)
LR = 2e-3
REG = 0.001
INPUT_SIZE = 3
NUM_CLASSES = 100
HIDDEN_SIZE = [128, 512, 512, 512, 512, 512]
NUM_EPOCHS = ...
BATCH_SIZE = ...
LR_DECAY = ...
TRAINING_SIZE = ...

# Percentage of the training data to be used as validation data
VAL_SIZE = ...
DROP_OUT = ...

In [ ]:
def get_cifar100_dataset(val_size=VAL_SIZE, batch_size=BATCH_SIZE):
    """
    Load and transform the CIFAR100 dataset. Make Validation set. Create dataloaders for
    train, test, validation sets.

    NOTES:
    1. DO NOT CHANGE THE CODE IN THIS FUNCTION. YOU MAY CHANGE THE BATCH_SIZE PARAM IF NEEDED.
    2. If you get an error related `num_workers`, you may change that parameter to a different value.

    :param val_size: size of the validation partition
    :param batch_size: number of samples in a batch
    :return: three data loaders and the set of possible classes
    """

    # the datasets.CIFAR getitem actually returns img in PIL format
    # no need to get to Tensor since we're working with our own model and not PyTorch
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.24703233, 0.24348505, 0.26158768))
                                    ])

    # Load the train_set and test_set from PyTorch, transform each sample to a flattened array
    train_set = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)
    classes = train_set.classes

    # Split data and define train_loader, test_loader, val_loader
    val_size = int(len(train_set) * val_size)
    train_size = len(train_set) - val_size
    train_set, val_set = random_split(train_set, [train_size, val_size])

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set),
                                              shuffle=False, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=val_size,
                                             shuffle=False, num_workers=2)

    return train_loader, test_loader, val_loader, classes

In [ ]:
train_loader, test_loader, val_loader, classes = get_cifar100_dataset(val_size=0.1, batch_size=200)

In [ ]:
# TODO: Implement the class `ConvNet`

In [ ]:
# TODO: Implement the forward pass computations

In [ ]:
# TODO: Implement the training function

In [ ]:
# TODO: Implement the evaluation function

## Bonus CNNs and NLP
Research the web and discuss how CNNs can be applied to NLP tasks:   
1. How is the input defined?
2. What advantages do CNNs have over fully connected NNs?  
3. For an RGB image input there are normally three channels - one for each color. What different channels can we consider given a language input?